In [7]:
from IPython.html import widgets

def update(name, value):
    d3sankeyExample.links[0]['value'] = value
    d3sankeyExample.send_state()

w = widgets.FloatSlider()
w.on_trait_change(update, 'value')
widgets.VBox([w, d3sankeyExample])

C:\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: DeprecationWarning: on_trait_change is deprecated: use observe instead


NameError: name 'd3sankeyExample' is not defined